In [1]:
# ########################################################################### #
#    Copyright (c) 2019-2020, California Institute of Technology.
#    All rights reserved.  Based on Government Sponsored Research under
#    contracts NNN12AA01C, NAS7-1407 and/or NAS7-03001.
#
#    Redistribution and use in source and binary forms, with or without
#    modification, are permitted provided that the following conditions
#    are met:
#      1. Redistributions of source code must retain the above copyright
#         notice, this list of conditions and the following disclaimer.
#      2. Redistributions in binary form must reproduce the above copyright
#         notice, this list of conditions and the following disclaimer in
#         the documentation and/or other materials provided with the
#         distribution.
#      3. Neither the name of the California Institute of
#         Technology (Caltech), its operating division the Jet Propulsion
#         Laboratory (JPL), the National Aeronautics and Space
#         Administration (NASA), nor the names of its contributors may be
#         used to endorse or promote products derived from this software
#         without specific prior written permission.
#
#    THIS SOFTWARE IS PROVIDED BY THE COPYRIGHT HOLDERS AND CONTRIBUTORS
#    "AS IS" AND ANY EXPRESS OR IMPLIED WARRANTIES, INCLUDING, BUT NOT
#    LIMITED TO, THE IMPLIED WARRANTIES OF MERCHANTABILITY AND FITNESS FOR
#    A PARTICULAR PURPOSE ARE DISCLAIMED. IN NO EVENT SHALL THE CALIFORNIA
#    INSTITUTE OF TECHNOLOGY BE LIABLE FOR ANY DIRECT, INDIRECT, INCIDENTAL,
#    SPECIAL, EXEMPLARY, OR CONSEQUENTIAL DAMAGES (INCLUDING, BUT NOT LIMITED
#    TO, PROCUREMENT OF SUBSTITUTE GOODS OR SERVICES; LOSS OF USE, DATA, OR
#    PROFITS; OR BUSINESS INTERRUPTION) HOWEVER CAUSED AND ON ANY THEORY OF
#    LIABILITY, WHETHER IN CONTRACT, STRICT LIABILITY, OR TORT (INCLUDING
#    NEGLIGENCE OR OTHERWISE) ARISING IN ANY WAY OUT OF THE USE OF THIS
#    SOFTWARE, EVEN IF ADVISED OF THE POSSIBILITY OF SUCH DAMAGE.
#
# ########################################################################### #
#    EXOplanet Transit Interpretation Code (EXOTIC)
#    # NOTE: See companion file version.py for version info.
# ########################################################################### #
!pip3 install -r ../requirements.txt
import json
import param
import panel as pn
import pandas as pd
import datetime as dt
from astropy.io import fits 
from io import BytesIO
import holoviews as hv
from holoviews import opts

pn.extension('ace','bokeh')

from exotic.exotic import NASAExoplanetArchive

inits_path = "../inits.json"
with open(inits_path) as json_file:
    data = json.load(json_file)


ERROR: Could not find a version that satisfies the requirement exotic~=0.23.2 (from -r ../requirements.txt (line 2)) (from versions: none)
ERROR: No matching distribution found for exotic~=0.23.2 (from -r ../requirements.txt (line 2))
You should consider upgrading via the '/Library/Frameworks/Python.framework/Versions/3.6/bin/python3.6 -m pip install --upgrade pip' command.


Created TAP+ (v1.2.1) - Connection:
	Host: gea.esac.esa.int
	Use HTTPS: True
	Port: 443
	SSL Port: 443
Created TAP+ (v1.2.1) - Connection:
	Host: geadata.esac.esa.int
	Use HTTPS: True
	Port: 443
	SSL Port: 443


ModuleNotFoundError: No module named 'exotic.exotic'; 'exotic' is not a package

In [ ]:
indeterminate = pn.widgets.Progress(name='Indeterminate Progress', active=False, width=1)

def anim_on():
    indeterminate.width=200 # TODO: enable
    indeterminate.active = True
    
def anim_off():
    indeterminate.width=1 # TODO: disable
    indeterminate.active = False

    
class ActionExample(param.Parameterized):
    """
    Demonstrates how to use param.Action to trigger an update.
    """

    planet = param.String(default="")
    
    action = param.Action(lambda x: x.param.trigger('action'), label='Resolve!')
        
    @param.depends('action')
    def resolve_target(self):
        if self.planet:
            
            # gui animations
            anim_on()
            
            # search archive
            nea_obj = NASAExoplanetArchive(planet=self.planet)
            if nea_obj.resolve_name():
                target, candidate, pl_dict = nea_obj.planet_info()
                anim_off()
                return pl_dict
            else:
                anim_off()
                return "Not found"
        else:
            return ""

action_example = ActionExample()
status = pn.WidgetBox(action_example.resolve_target, width=300)

pn.Column(
    '## Search for a target in the NASA Exoplanet Archive',
    pn.Row(
        pn.Column(
            pn.panel(action_example.param, show_labels=False, show_name=False), 
            indeterminate
        ),
        status
    )
).servable()

In [ ]:
# TODO create load action 
user_editor = pn.widgets.Ace(value=json.dumps(data['user_info'], indent=4),
                             theme= 'monokai', width=750, height=225)

planet_editor = pn.widgets.Ace(value=json.dumps(data['planetary_parameters'], indent=4),
                             theme= 'monokai', width=750, height=225)

pn.Column(
    pn.Row("## File:",inits_path),

    pn.Column(
        pn.Column("## User Info", user_editor ),
        pn.Column("## Planet Info", planet_editor ),
    )
)

# TODO figure out how to save
# Check 

In [ ]:
# trying to get directory from 
from io import BytesIO

fileinput = pn.widgets.FileInput()

class ActionFileExample(param.Parameterized):
    
    action = param.Action(lambda x: x.param.trigger('action'), label='Load Data!')
    json_output = {}
    
    @param.depends('action')
    def fileAction(self):
        if fileinput.filename:            
            # load image from byte array
            hdu = fits.open( BytesIO(fileinput.value) )
            img = hdu[0].data
            
            # clean up header objects
            dheader = dict(hdu[0].header)
            del(dheader[''])
            dheader['COMMENT'] = str(dheader['COMMENT'])
            dheader['FILENAME'] = fileinput.filename
            # return JSON view
            self.json_output = dheader
            
            # return header and image
            return pn.Row(
                pn.pane.JSON(self.json_output, width=500, height=20*len(dheader.keys())),
                hv.Image(img).opts(xaxis=None,yaxis=None)
            ) 
        else:
            self.json_output ={"dude":"nice"}
            return  pn.pane.JSON(self.json_output, width=300)

action_example = ActionFileExample()
status = pn.WidgetBox(action_example.fileAction)

pn.Column( 
    '### Select a FITS file:',
    fileinput,
    pn.panel( action_example.param, show_name=False), 
    status
).servable()

In [ ]:
action_example.json_output

In [ ]:
# doesn't work as intended: 
    
inits_path = "inits.json"
with open(inits_path) as json_file:
    data = json.load(json_file)

class Planet_Info(param.Parameterized):
    
    def __init__(self, data, **kwargs):
        for k in data['planetary_parameters']:
            
            new_key = ''.join([ki for ki in k.split(' ') if '(' not in ki])
            new_key = new_key.replace("'","")

            if isinstance(data['planetary_parameters'][k], str):
                setattr(self,new_key, param.String(default=data['planetary_parameters'][k], doc=k))
            elif isinstance(data['planetary_parameters'][k], float):
                setattr(self,new_key, param.Number(default=data['planetary_parameters'][k], doc=k))
            
        super().__init__(**kwargs)

pn.Row( Planet_Info(data))